# Final Model Evaluation & Explainability

This notebook evaluates the final tuned model selected and explains its behavior
using SHAP interpretability techniques.

Since the problem involves medical classification (malignant vs benign tumors),
model transparency is essential. We analyze:

- Generalization metrics (Accuracy, Precision, Recall, F1, ROC-AUC)
- Confusion matrix and classification report
- ROC and Precision–Recall curves
- SHAP global feature importance
- SHAP local explanations for individual predictions

This step completes the analytical component of the project.

In [2]:
import sys
sys.path.append("..")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, auc, classification_report,
    confusion_matrix, precision_recall_curve
)

import joblib
import shap

from src.config import SELECTED_FEATURES